In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [ ]:
db_bd = sqlite3.connect("orf_features.sqlite")

with db_bd:
    db_bd.execute('ATTACH DATABASE "features.sqlite" AS features')
    db_bd.execute('ATTACH DATABASE "sgd_go.sqlite" AS sgd_go')
    db_bd.execute('ATTACH DATABASE "fitness.sqlite" AS fitness')

In [ ]:
print(pd.read_sql_query("SELECT * FROM ohnologs limit 5",db_bd))
print(pd.read_sql_query("SELECT Systematic,GO FROM sgd_go_map limit 5",db_bd))
print(pd.read_sql_query("SELECT * FROM n_positive limit 5",db_bd))

In [ ]:
with db_bd:
    db_bd.execute("\
        CREATE TABLE membrane_table AS \
        SELECT * FROM \
            ( SELECT YORF AS Systematic FROM yorf_list ) \
        LEFT OUTER JOIN \
            ( SELECT DISTINCT Systematic,'membrane' AS Membrane FROM sgd_go_map \
                WHERE GO_desc=='membrane' ) \
        USING (Systematic) \
        ")

    db_bd.execute("\
        UPDATE membrane_table \
        SET Membrane = 'non-membrane' WHERE Membrane IS NULL \
        ")

pd.read_sql_query("\
        SELECT * FROM membrane_table LIMIT 10\
    ",db_bd)

In [ ]:
with db_bd:
    db_bd.execute("\
        CREATE TABLE main.per_orf AS \
        SELECT * FROM ( SELECT * FROM yorf_list) \
        LEFT OUTER JOIN \
            costanza2016 \
            USING (YORF) \
        LEFT OUTER JOIN \
            (SELECT YORF, MeanMol, MedianMol FROM ho2018) \
            USING (YORF) \
        LEFT OUTER JOIN \
            (SELECT * FROM biogrid_ptms) \
            USING (YORF) \
        LEFT OUTER JOIN \
            (SELECT *,Systematic AS YORF FROM membrane_table) \
            USING (YORF) \
        ")
    db_bd.execute("\
        UPDATE per_orf \
            SET `Number.of.complexes` = 0 \
            WHERE `Number.of.complexes` IS NULL \
        ")

In [ ]:
pd.read_sql_query("SELECT * FROM per_orf LIMIT 10",db_bd)

In [ ]:
pd.read_sql_query("SELECT * FROM per_orf LIMIT 10",db_bd)

In [ ]:
db_bd.close()